# Using continous and categorical only from transition matrices the category was obtained to filter data by category, but transi

In [126]:
import pandas as pd
from sklearn.cluster import KMeans
import seaborn as sns
from matplotlib import pyplot as plt 
import math
from ipynb.fs.full.Utils import createKDeplot, loadDataFrames, dropFeaturesWithNoVariance, loadCoverGroups
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy import stats
import dask_ml.cluster
from ipynb.fs.full.Pipeline import removeConstantFeatures, applyBoxCoxAllFeatures, centerAndScale, applyEuclideanDistance, getIndexAndVector, applyBoxCoxFeatures,applyJaccard, getEuclideanJaccardDf 

In [127]:
spotifyFeatures, transitionMatrices, df = loadDataFrames()

In [128]:
ids = transitionMatrices.index.tolist()

In [129]:
transitionMatrices['id'] = ids

In [130]:
df = transitionMatrices

In [131]:
df

,0,1,2,3,4,5,6,7,8,9,...,1286,1287,1288,1290,1291,1292,1293,1294,1295,id
id,,,,,,,,,,,,,,,,,,,,,
23,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,23
105,0.185185,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,105
118,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.044944,0.000000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,118
120,0.029412,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,120
292,0.013889,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.013889,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6002496,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,6002496
6002497,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,6002497
6002498,0.000000,0.015385,0.0,0.015385,0.0,0.015385,0.030769,0.000000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,6002498


In [132]:
## Removing unamed and other unnecessary columns
df.reset_index(drop=True, inplace=True)
df = df.sort_values('id')
df.id = df.id.astype('int64')
# Saving idChords
ids = df.id
df = df.set_index('id')
df.reset_index(drop=True, inplace=True)

In [133]:
originalSongId = 180849 # The Scientist Original
originalSongIndex,originalSongVector = getIndexAndVector(originalSongId, ids, df)
coverSongId = 5000011 # The Scientist Original
coverSongIndex,coverSongVector = getIndexAndVector(coverSongId, ids, df)

In [134]:
ranksDFEuclidean  = applyEuclideanDistance(df, ids, originalSongVector)
ranksDFEuclidean

,id,rank
0,180849,0.000000
1,2846348,0.158068
2,1186397,0.176593
3,1148352,0.177127
4,1168573,0.185495
...,...,...
5975,752135,0.791540
5976,538427,0.802009
5977,2354751,0.802108
5978,649262,0.815992


In [135]:
ranksDFEuclidean.loc[(ranksDFEuclidean.id > 5000000) & (ranksDFEuclidean.id <= 5900000)]

,id,rank
28,5000007,0.238091
51,5000019,0.263030
56,5000018,0.267234
138,5000020,0.301116
173,5000008,0.310970
260,5000011,0.325204
271,5000012,0.326780
281,5000017,0.327370
304,5000013,0.330925
437,5000001,0.345503



### Using only 20% of the most similar songs in terms of chord progressions to recommend songs

In [136]:
percentage = 20
upperLimit = math.floor((len(ranksDFEuclidean) * percentage) / 100)

In [137]:
ids = ranksDFEuclidean.iloc[0:upperLimit,:].id

In [138]:
df = pd.read_csv("./data/unified/05-allDataContinousCategoricalTransition.csv")
df = df.drop(columns=['Unnamed: 0', 'index'])

In [139]:
df

,id,danceability,energy,speechiness,acousticness,liveness,valence,tempo,0,1,...,mood_5,mood_6,mood_7,mood_8,mood_9,mood_10,mood_11,instrumentalness_cat_0,instrumentalness_cat_1,instrumentalness_cat_2
0,23,0.494,0.756,0.0432,0.001400,0.1640,0.589,133.337,0.000000,0.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,105,0.582,0.744,0.0336,0.000458,0.0405,0.674,133.344,0.185185,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,118,0.614,0.755,0.0276,0.016200,0.0831,0.645,92.622,0.000000,0.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120,0.319,0.807,0.0388,0.001040,0.1310,0.601,118.097,0.029412,0.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,292,0.496,0.527,0.0339,0.195000,0.1380,0.283,155.932,0.013889,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5972,6002496,0.509,0.537,0.0384,0.347000,0.0805,0.311,170.030,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5973,6002497,0.725,0.583,0.0427,0.300000,0.0665,0.762,79.984,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
5974,6002498,0.399,0.787,0.0499,0.019700,0.0685,0.572,117.089,0.000000,0.015385,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5975,6002499,0.376,0.435,0.0264,0.513000,0.1380,0.304,80.002,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [140]:
df = df[df['id'].isin(ids.tolist())]

In [141]:
df

,id,danceability,energy,speechiness,acousticness,liveness,valence,tempo,0,1,...,mood_5,mood_6,mood_7,mood_8,mood_9,mood_10,mood_11,instrumentalness_cat_0,instrumentalness_cat_1,instrumentalness_cat_2
1,105,0.582,0.744,0.0336,0.000458,0.0405,0.674,133.344,0.185185,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
5,407,0.577,0.890,0.0345,0.000013,0.0954,0.643,132.397,0.000000,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,469,0.664,0.787,0.0239,0.038000,0.1930,0.585,102.952,0.007634,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
15,830,0.526,0.665,0.0243,0.407000,0.1690,0.285,158.493,0.181818,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
28,3165,0.527,0.418,0.0277,0.142000,0.1390,0.217,129.517,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5942,6002464,0.434,0.897,0.0695,0.012400,0.0772,0.376,135.540,0.020833,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5944,6002466,0.672,0.594,0.0789,0.062600,0.1520,0.158,171.854,0.333333,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5964,6002488,0.760,0.838,0.0510,0.390000,0.1660,0.863,159.938,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
5971,6002495,0.555,0.578,0.0277,0.113000,0.0747,0.333,83.382,0.000000,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [142]:
## Removing unamed and other unnecessary columns
df.reset_index(drop=True, inplace=True)
df = df.sort_values('id')
df.id = df.id.astype('int64')
# Saving idChords
ids = df.id
df = df.set_index('id')
df.reset_index(drop=True, inplace=True)

In [143]:
df

,danceability,energy,speechiness,acousticness,liveness,valence,tempo,0,1,2,...,mood_5,mood_6,mood_7,mood_8,mood_9,mood_10,mood_11,instrumentalness_cat_0,instrumentalness_cat_1,instrumentalness_cat_2
0,0.582,0.744,0.0336,0.000458,0.0405,0.674,133.344,0.185185,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,0.577,0.890,0.0345,0.000013,0.0954,0.643,132.397,0.000000,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.664,0.787,0.0239,0.038000,0.1930,0.585,102.952,0.007634,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.526,0.665,0.0243,0.407000,0.1690,0.285,158.493,0.181818,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.527,0.418,0.0277,0.142000,0.1390,0.217,129.517,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191,0.434,0.897,0.0695,0.012400,0.0772,0.376,135.540,0.020833,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1192,0.672,0.594,0.0789,0.062600,0.1520,0.158,171.854,0.333333,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1193,0.760,0.838,0.0510,0.390000,0.1660,0.863,159.938,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1194,0.555,0.578,0.0277,0.113000,0.0747,0.333,83.382,0.000000,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## Pipeline

In [147]:
dfEuclidean, dfJaccard = getEuclideanJaccardDf(df)

In [148]:
dfEuclidean

,0,1,10,100,1000,1001,1002,1003,1004,1005,...,997,998,999,acousticness,danceability,energy,liveness,speechiness,tempo,valence
0,0.185185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000458,0.582,0.744,0.0405,0.0336,133.344,0.674
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000013,0.577,0.890,0.0954,0.0345,132.397,0.643
2,0.007634,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.038000,0.664,0.787,0.1930,0.0239,102.952,0.585
3,0.181818,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.407000,0.526,0.665,0.1690,0.0243,158.493,0.285
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.142000,0.527,0.418,0.1390,0.0277,129.517,0.217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191,0.020833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.012400,0.434,0.897,0.0772,0.0695,135.540,0.376
1192,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.062600,0.672,0.594,0.1520,0.0789,171.854,0.158
1193,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.390000,0.760,0.838,0.1660,0.0510,159.938,0.863
1194,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.113000,0.555,0.578,0.0747,0.0277,83.382,0.333


### Get Rankings Euclidean

In [150]:
dfEuclidean = dfEuclidean.copy()
dfEuclidean = removeConstantFeatures(dfEuclidean)
dfEuclidean= applyBoxCoxAllFeatures(dfEuclidean)
dfEuclidean = centerAndScale(dfEuclidean)

### Original Song & Cover Song

In [151]:
originalSongId = 180849 # The Scientist Original
originalSongIndex,originalSongVector = getIndexAndVector(originalSongId, ids, dfEuclidean)
coverSongId = 5000011 # The Scientist Original
coverSongIndex,coverSongVector = getIndexAndVector(coverSongId, ids, dfEuclidean)

### Get Rankings Euclidean

In [152]:
ranksDFEuclidean  = applyEuclideanDistance(dfEuclidean, ids, originalSongVector)
ranksDFEuclidean

,id,rank
0,180849,0.000000
1,748267,15.591743
2,2065851,15.811639
3,6001027,15.839478
4,2689908,15.901581
...,...,...
1191,6000798,139.364056
1192,6000742,174.781528
1193,6000790,180.056534
1194,6000792,197.170372


In [153]:
ranksDFEuclidean[ranksDFEuclidean.id == 180849]

,id,rank
0,180849,0.0


In [154]:
ranksDFEuclidean[ranksDFEuclidean['id']==coverSongId]

,id,rank
145,5000011,16.905711


In [155]:
ranksDFEuclidean.loc[(ranksDFEuclidean['id'] > 5000000) & (ranksDFEuclidean['id'] < 5900000)]

,id,rank
14,5000019,16.244260
19,5000017,16.294221
117,5000013,16.829004
145,5000011,16.905711
209,5000012,17.094336
503,5000001,18.359178
551,5000018,18.728653
825,5000007,23.981770
857,5000008,25.358127
913,5000020,27.580543


### Get Rankings Jaccard

In [159]:
originalSongId = 180849 # The Scientist Original
originalSongIndex,originalSongVector = getIndexAndVector(originalSongId, ids, dfJaccard)
coverSongId = 5000011 # The Scientist Original
coverSongIndex,coverSongVector = getIndexAndVector(coverSongId, ids, dfJaccard)

In [160]:
ranksDFJaccard  = applyJaccard(dfJaccard, ids, originalSongVector)

In [161]:
ranksDFJaccard

,id,rank
0,1029726,0.000000
1,180849,0.000000
2,6001027,0.000000
3,466649,0.000000
4,1088889,0.333333
...,...,...
1191,39044,1.000000
1192,698704,1.000000
1193,84885,1.000000
1194,821402,1.000000


In [162]:
ranksDFJaccard[ranksDFJaccard['id']==coverSongId]

,id,rank
676,5000011,0.75


In [163]:
ranksDFJaccard[ranksDFJaccard['id']==coverSongId]

,id,rank
145,5000011,16.905711


In [164]:
ranksDFJaccard[ranksDFJaccard['id'] > 5000000]

,id,rank
3,6001027,15.839478
14,5000019,16.244260
19,5000017,16.294221
29,6000372,16.366425
84,6001418,16.744682
...,...,...
1191,6000798,139.364056
1192,6000742,174.781528
1193,6000790,180.056534
1194,6000792,197.170372


### Experiment to unify rangs

In [165]:
ranks = pd.merge(ranksDFJaccard,ranksDFEuclidean, on='id')

In [166]:
ranks['new_rank'] = ranks['rank_x'] + ranks['rank_y']

In [167]:
ranks = ranks.sort_values(by='new_rank')
ranks.reset_index(drop=True, inplace=True)

In [171]:
ranks

,id,rank_x,rank_y,new_rank
0,180849,0.000000,0.000000,0.000000
1,6001027,0.000000,15.839478,15.839478
2,2065851,0.333333,15.811639,16.144973
3,748267,0.571429,15.591743,16.163172
4,2689908,0.571429,15.901581,16.473010
...,...,...,...,...
1191,6000798,0.750000,139.364056,140.114056
1192,6000742,0.750000,174.781528,175.531528
1193,6000790,0.750000,180.056534,180.806534
1194,6000792,0.888889,197.170372,198.059261


In [168]:
ranks[ranks.id > 5000000]

,id,rank_x,rank_y,new_rank
1,6001027,0.000000,15.839478,15.839478
5,5000019,0.333333,16.244260,16.577593
7,5000017,0.333333,16.294221,16.627555
23,6000372,0.571429,16.366425,16.937853
83,6000268,0.571429,16.781984,17.353413
...,...,...,...,...
1191,6000798,0.750000,139.364056,140.114056
1192,6000742,0.750000,174.781528,175.531528
1193,6000790,0.750000,180.056534,180.806534
1194,6000792,0.888889,197.170372,198.059261


In [169]:
ranks.head(10)

,id,rank_x,rank_y,new_rank
0,180849,0.000000,0.000000,0.000000
1,6001027,0.000000,15.839478,15.839478
2,2065851,0.333333,15.811639,16.144973
3,748267,0.571429,15.591743,16.163172
4,2689908,0.571429,15.901581,16.473010
5,5000019,0.333333,16.244260,16.577593
6,1909308,0.571429,16.055998,16.627426
7,5000017,0.333333,16.294221,16.627555
8,1473649,0.333333,16.420997,16.754330
9,872773,0.333333,16.439907,16.773241


In [170]:
ranks.loc[(ranks.id > 5000000) & (ranks.id < 5900000)]

,id,rank_x,rank_y,new_rank
5,5000019,0.333333,16.244260,16.577593
7,5000017,0.333333,16.294221,16.627555
95,5000013,0.571429,16.829004,17.400432
171,5000011,0.750000,16.905711,17.655711
235,5000012,0.750000,17.094336,17.844336
452,5000001,0.333333,18.359178,18.692511
537,5000018,0.571429,18.728653,19.300082
826,5000007,0.750000,23.981770,24.731770
856,5000008,0.571429,25.358127,25.929555
904,5000020,0.333333,27.580543,27.913876
